## Computer

This notebook demonstrates how to use Computer to operate a Lume sandbox programmatically on Apple Silicon macOS systems.

### Installation

In [2]:
!pip uninstall -y cua-computer

In [ ]:
!pip install cua-computer

In [ ]:
# If locally installed, use this instead:
import os

os.chdir('../libs/computer')
!poetry install
!poetry build

!pip uninstall cua-computer -y
!pip install ./dist/cua_computer-0.1.0-py3-none-any.whl --force-reinstall

## Lume daemon

While a `lume` binary is included with Computer, we recommend installing the standalone version with brew, and starting the lume daemon service before running Computer. Refer to [../libs/lume/README.md](../libs/lume/README.md) for more details on lume cli.

For installing the standalone lume binary, run the following command from a terminal:

```bash
sudo /bin/bash -c "$(curl -fsSL https://raw.githubusercontent.com/trycua/cua/main/libs/lume/scripts/install.sh)"
```

Start the lume daemon service (from another terminal):

```bash
lume serve
```

Pull the latest pre-built macos-sequoia-cua image. This image, based on macOS Sequoia, contains all dependencies needed to be controlled from the Computer interface.

In [ ]:
!lume pull macos-sequoia-cua:latest

The initial image download of thew macos-sequoia-cua image requires 80GB of storage space. However, after the first run, the image size reduces to around 20GB. Thanks to macOS's sparse file system, VM disk space is allocated dynamically - while VMs may show a total size of 50GB, they typically only consume about 20GB of physical disk space.

Sandbox are stored in `~/.lume`, and locally cached images are stored in `~/.lume/cache`.


You can always see the list of downloaded VM images with:

In [4]:
!lume ls

name                              os      cpu     memory  disk            display     status          ip              vnc                                               
macos-sequoia-cua_latest          macOS   12      16.00G  64.5GB/80.0GB   1024x768    running         192.168.64.78   vnc://:kind-forest-zulu-island@127.0.0.1:56085    


## Testing the sandbox

Once downloaded, you can run the sandbox to test everything is working:

```bash
lume run macos-sequoia-cua:latest
```

You can add additional software and tools to the sandbox - these changes will be saved in the VM disk.

## Initialize a Computer instance

Computer allows you to create and control a virtual sandbox instances from your host on Apple Silicon. Here's a basic example:

In [2]:
from computer import Computer

You can either use the async context manager or initialize the Computer instance directly.

With the async context manager:

In [ ]:
async with Computer(
    # name="my_vm", # optional, in case you want to use any other VM created using lume
    display="1024x768",
    memory="8GB",
    cpu="4",
    os="macos"
) as computer:
    await computer.run()
    # ... do something with the computer interface

Computer API Server not ready yet. Will retry automatically.


Direct initialization:

In [ ]:
computer = Computer(
    display="1024x768",
    memory="8GB",
    cpu="4",
    os="macos"
)

await computer.run()
# ... do something with the computer interface

The Computer instance requires a Lume server for communication. Here's how it works:

1. First, it attempts to connect to any existing Lume server running on port 3000
2. If no Lume server is available, it automatically starts a new one via [lume serve](https://github.com/trycua/lume/?tab=readme-ov-file#local-api-server)

The sandbox's lifecycle is tied to the Lume server:
- If Computer started the Lume server itself, the server will be terminated when Computer stops
- If Computer connected to a pre-existing server, that server remains running after Computer stops

If you have scenarios where you need to run multiple sandboxes in parallel, we recommend first starting the Lume server separately with `lume serve` (refer to [Lume](https://github.com/trycua/lume/?tab=readme-ov-file#install) on how to install) and then having each Computer instance connect to it.

In order to execute commands targeting the sandbox, the Computer interface communicates through websockets to a Fast API server running on the sandbox.

### Cursor

In [4]:
# Move and click
await computer.interface.move_cursor(100, 100)
await computer.interface.left_click()
await computer.interface.right_click(300, 300)
await computer.interface.double_click(400, 400)

# Drag operations
await computer.interface.drag_to(500, 500, duration=1.0)

# Get cursor position
cursor_pos = await computer.interface.get_cursor_position()

### Keyboard

In [9]:
# Type text
await computer.interface.type_text("Hello, World!")

# Press individual keys
await computer.interface.press_key("enter")
await computer.interface.press_key("escape")

# Use hotkeys
await computer.interface.hotkey("command", "c")  # Command+C on macOS

### Screen

In [10]:
# Get screen dimensions
screen_size = await computer.interface.get_screen_size()

# Take basic screenshot
screenshot = await computer.interface.screenshot()
with open("screenshot.png", "wb") as f:
    f.write(screenshot)

### Clipboard

In [ ]:
# Set clipboard content
await computer.interface.set_clipboard("Text to copy")

# Get clipboard content
clipboard_content = await computer.interface.copy_to_clipboard()

### File System Operations

In [ ]:
# Check file/directory existence
file_exists = await computer.interface.file_exists("/path/to/file.txt")
dir_exists = await computer.interface.directory_exists("/path/to/directory")

# Run shell commands
stdout, stderr = await computer.interface.run_command("ls -la")

### Accessibility

In [8]:
# Get accessibility tree
accessibility_tree = await computer.interface.get_accessibility_tree()

### Sharing a directory with the sandbox

You can share a directory with the sandbox by passing a list of absolute paths to the `shared_directories` argument when initializing the Computer instance.

In [ ]:
computer = Computer(
    display="1024x768",
    memory="4GB",
    cpu="2",
    os="macos",
    shared_directories=["/absolute/path/to/directory"]
)

### Using Host Computer

For local development and testing purposes, you can run the Computer API server on the same host machine and target it instead:

In [ ]:
computer = Computer(
    display="1024x768",
    memory="4GB",
    cpu="2",
    os="macos",
    use_host_computer_server=True
)

### Examples

In [3]:
from computer.computer import Computer

In [ ]:
async with Computer(
    display="1024x768",
    memory="4GB",
    cpu="2",
    os="macos"
) as computer:
    await computer.run()
    res = await computer.interface.run_command("ls -a")

    # Get screen dimensions
    screen_size = await computer.interface.get_screen_size()

    # Move and click
    await computer.interface.move_cursor(100, 100)
    await computer.interface.left_click()
    await computer.interface.right_click(300, 300)
    await computer.interface.double_click(400, 400)

    # Drag operations
    await computer.interface.drag_to(500, 500, duration=1.0)

    # Get cursor position
    cursor_pos = await computer.interface.get_cursor_position()

    # Your automation code here
    await computer.stop()